In [1]:
import os
import json
import logging
import numpy as np
from tqdm import tqdm
from utils.load_dataset import load_dataset

with open("./config.json") as f:
    config = json.load(f)

logger = logging.getLogger("")
dataset_name = 'cifar100'
arch = 'resnet18'
num_augs = 2

dataset = load_dataset(
    dataset=dataset_name,
    train_batch_size=128,
    test_batch_size=128,
    val_split=0.0,
    augment=False,
    shuffle=False,
    root_path=config['data_dir'],
    random_seed=0,
    mean=[0, 0, 0],
    std=[1, 1, 1],
    logger=logger,
)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [2]:
def get_scores_and_masks(seeds, dataset, method, num_augs):
    h = 0.001
    num_seeds = len(seeds)
    aug_scores_file_name = {
        'curv_zo' : "curvature_scores_zo_{}_{}_{}_{}_tid{}.pt",
        'curv' : "curvature_scores_{}_{}_{}_{}_tid{}.pt",
        'prob' : "prob_{}_{}_{}_tid{}.pt",
        'loss' : "losses_{}_{}_{}_tid{}.pt",
        'mentr' : "m_entropy_{}_{}_{}_tid{}.pt",
        'loss_g' : "loss_g_{}_{}_{}_tid{}.pt",
    }

    score_seeds_v_seed = np.zeros((num_augs, dataset.train_length, num_seeds))
    mask_seeds_v_seed = np.zeros((dataset.train_length, num_seeds))
    start_seed = seeds[0]
    for exp_idx in seeds:
        array_idx = exp_idx - start_seed
        augs = 1 if method == 'mentr' else num_augs
        for aug_idx in range(augs):
            if 'curv' in aug_scores_file_name[method]:
                file_name = aug_scores_file_name[method].format(dataset.name, arch, exp_idx, h, aug_idx)
            else:
                file_name = aug_scores_file_name[method].format(dataset.name, arch, exp_idx, aug_idx)

            absolute_file_name = os.path.join(config['precomputed_scores_dir'], dataset.name, file_name)
            scores = np.load(absolute_file_name)['data']
            score_seeds_v_seed[aug_idx, :, array_idx] = scores

        if dataset == 'imagenet':
            array = np.load(f'/path/imagenet-resnet50/{0.7}/{exp_idx}/aux_arrays.npz')
            mask_idxs = array['subsample_idx']
            mask_seeds_v_seed[mask_idxs, array_idx] = 1
        else:
            mask_idxs = np.load(os.path.join(config['subset_idxs_dir'], f"{exp_idx}.npy"))
            mask_seeds_v_seed[mask_idxs, array_idx] = 1

    return score_seeds_v_seed, mask_seeds_v_seed

def get_score_dist_params(scores, masks, num_augs):
    means_in = []
    means_out = []
    var_in = []
    var_out = []
    for aug in range(num_augs):
        in_scores = np.where(masks == 1, scores[aug], np.NaN)
        out_scores = np.where(masks == 0, scores[aug], np.NaN)
        means_in.append(np.nanmean(in_scores, axis=1))
        means_out.append(np.nanmean(out_scores, axis=1))
        var_in.append(np.nanvar(in_scores, axis=1))
        var_out.append(np.nanvar(out_scores, axis=1))

    return {
        'means_in': np.row_stack(means_in),
        'means_out': np.row_stack(means_out),
        'var_in': np.row_stack(var_in),
        'var_out': np.row_stack(var_out)
    }


In [3]:
scores = {}
masks = {}
dist_params = {}
train_end_seed = 64
train_seeds = list(range(train_end_seed))
test_seeds = list(range(train_end_seed, train_end_seed + 3))
eps = 1e-22

In [4]:
def log_err(x):
    return np.log(x + eps)

def logit_scale(x):
    x = x / (1 - x + eps)
    return log_err(x)

def nll(curv_score, mean, var):
    ll = - ( ( (curv_score - mean)**2) / (2 * (var ** 2) ) ) -0.5 * np.log(var ** 2) - 0.5 * np.log(2 * np.pi)
    return -ll

def likelihood(curv_score, mean, var):
    nll = - ( ( (curv_score - mean)**2) / (2 * (var ** 2) ) ) - 0.5 * np.log(var ** 2) - 0.5 * np.log(2 * np.pi)
    likelihood_val = np.exp(nll)
    return likelihood_val

def get_likelihood_ratio(test_scores, dist_params, num_augs, train_scores, train_masks):
    in_likelihood = np.zeros((test_scores.shape[1]))
    out_likelihood = np.zeros_like(in_likelihood)
    for aug in range(num_augs):
        test_score_aug = test_scores[aug, :, 0]
        in_likelihood += likelihood(
            test_score_aug, 
            dist_params['means_in'][aug], 
            dist_params['var_in'][aug] + eps)

        out_likelihood += likelihood(
            test_score_aug, 
            dist_params['means_out'][aug], 
            dist_params['var_out'][aug] + eps)

    likelihood_ratio = in_likelihood / (out_likelihood + eps)
    return likelihood_ratio

def get_nll_ratio(test_scores, dist_params, num_augs, train_scores, train_masks):
    in_likelihood = np.zeros((test_scores.shape[1]))
    out_likelihood = np.zeros_like(in_likelihood)
    for aug in range(num_augs):
        test_score_aug = test_scores[aug, :, 0]
        in_likelihood += nll(
            test_score_aug, 
            dist_params['means_in'][aug], 
            dist_params['var_in'][aug] + 1e-32)

        out_likelihood += nll(
            test_score_aug, 
            dist_params['means_out'][aug], 
            dist_params['var_in'][aug] + 1e-32)

    likelihood_ratio = in_likelihood - out_likelihood
    return -likelihood_ratio

def get_nll_ratio_c(test_scores, dist_params, num_augs, train_scores, train_masks):
    in_likelihood = np.zeros((test_scores.shape[1]))
    out_likelihood = np.zeros_like(in_likelihood)
    for aug in range(num_augs):
        test_score_aug = test_scores[aug, :, 0]
        in_likelihood += nll(
            test_score_aug, 
            dist_params['means_in'][aug], 
            1)

        out_likelihood += nll(
            test_score_aug, 
            dist_params['means_out'][aug], 
            1)

    likelihood_ratio = in_likelihood - out_likelihood
    return -likelihood_ratio

def get_identity(test_scores, dist_params, num_augs, train_scores, train_masks):
    return -test_scores[0]

def get_mast(test_scores, dist_params, num_augs, train_scores, train_masks):
    aug = 0
    threshold = (dist_params['means_in'][aug] +  dist_params['means_out'][aug]) / 2 
    return -(test_scores[aug, :, 0] - threshold)

def get_mast_offline(test_scores, dist_params, num_augs, train_scores, train_masks):
    aug = 0
    threshold = dist_params['means_out'][aug]
    return -(test_scores[aug, :, 0] - threshold)

def get_ye_et_el_attack_r(test_scores, dist_params, num_augs, train_scores, train_masks):
    aug = 0
    alpha = 1e-4
    out_scores = np.where(train_masks == 0, train_scores[aug], np.NaN)
    threshold = np.nanpercentile(out_scores, alpha * 100, axis=1)
    return -(test_scores[aug, :, 0] - threshold)

def get_class_based(test_scores, dist_params, num_augs, train_scores, train_masks):
    # This is handled during roc calculations
    return -test_scores[0, :, 0]

def get_loss_count(test_scores, dist_params, num_augs, train_scores, train_masks):
    in_scores = np.where(train_masks == 1, train_scores[0], np.NaN)
    threshold = np.nanmean(in_scores)
    pred = (test_scores[:, :, 0] < threshold).sum(0) / num_augs
    return pred

methods = [
    ('curv_zo', log_err, get_nll_ratio, 'Curv ZO NLL', num_augs, train_seeds), 
    ('curv_zo', log_err, get_likelihood_ratio, 'Curv ZO LR', num_augs, train_seeds), 
    ('prob', logit_scale, get_likelihood_ratio, 'Carlini et al.', num_augs, train_seeds),
    ('loss', lambda x: x, get_identity, 'Yeom et al.', 1, [0]),
    ('loss', lambda x: x, get_mast, 'Sablayrolles et al.', 1, train_seeds),
    ('loss', lambda x: x, get_mast_offline, 'Watson et al.', 1, train_seeds),
    ('loss', lambda x: x, get_ye_et_el_attack_r, 'Ye et al.', 1, train_seeds),
    ('mentr', lambda x: x, get_class_based, 'Song et al.', 1, train_seeds),
]


In [5]:
for info_type, score_func, _, method_name, augs, seeds in tqdm(methods):
    if method_name in scores:
        continue
    
    score, mask = get_scores_and_masks(
        seeds=seeds, 
        dataset=dataset,
        method=info_type,
        num_augs=augs)

    score = score_func(score)
    params = get_score_dist_params(score, mask, augs)
    scores[method_name] = score
    masks[method_name] = mask
    dist_params[method_name] = params

 38%|███▊      | 3/8 [00:03<00:06,  1.33s/it]/tmp/ipykernel_5822/2255348333.py:47: RuntimeWarning: Mean of empty slice
  means_in.append(np.nanmean(in_scores, axis=1))
/tmp/ipykernel_5822/2255348333.py:48: RuntimeWarning: Mean of empty slice
  means_out.append(np.nanmean(out_scores, axis=1))
/tmp/ipykernel_5822/2255348333.py:49: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var_in.append(np.nanvar(in_scores, axis=1))
/tmp/ipykernel_5822/2255348333.py:50: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var_out.append(np.nanvar(out_scores, axis=1))
100%|██████████| 8/8 [00:06<00:00,  1.17it/s]


In [6]:
test_scores = {}
test_masks = {}

for info_type, score_func, _, method_name, augs, _ in tqdm(methods):
    test_score, test_mask = get_scores_and_masks(
        seeds=test_seeds, 
        dataset=dataset,
        method=info_type,
        num_augs=augs)

    test_scores[method_name] = score_func(test_score)
    test_masks[method_name] = test_mask


100%|██████████| 8/8 [00:00<00:00, 34.02it/s]


In [7]:
def find_tpr_at_fpr(tpr, fpr, target_fpr=1e-1):
    """
    Finds the TPR when FPR equals target_fpr using interpolation.

    Parameters:
    tpr (array): Array of true positive rates.
    fpr (array): Array of false positive rates.
    target_fpr (float): The target false positive rate. Default is 1e-1.

    Returns:
    float: Interpolated TPR at the target FPR.
    """
    # Ensure the arrays are numpy arrays
    tpr = np.array(tpr)
    fpr = np.array(fpr)

    # Use numpy's interpolation function
    tpr_at_target_fpr = np.interp(target_fpr, fpr, tpr)
    return tpr_at_target_fpr

In [8]:
from sklearn.metrics import roc_auc_score, roc_curve, balanced_accuracy_score

auroc_all = {}
bal_acc_all = {}
tpr_m1 = {}
tpr_m2 = {}
tpr_m3 = {}
tpr_m4 = {}

for test_seed_idx, test_seed in enumerate(test_seeds):
    print(f"=================== Seed {test_seed} ======================")
    for idx, (info_type, _, pred_func, method_name, augs, _) in enumerate(methods):
        y_pred = pred_func(
            test_scores[method_name][..., test_seed_idx, np.newaxis],
            dist_params[method_name],
            augs,
            scores[method_name],
            masks[method_name])

        y_true = test_masks[method_name][..., test_seed_idx, np.newaxis]
        if method_name == 'Song et al.':
            labels_file_name = os.path.join(
                config['precomputed_scores_dir'], 
                dataset.name, 
                f"true_labels_cifar100_{train_end_seed}.pt")
            labels = np.load(labels_file_name)['data']
            auc = 0 
            n_classes = dataset.num_classes
            balanced_accuracy = 0
            for label in range(n_classes):
                indices = np.where(labels == label)[0]
                # Compute ROC curve for each class
                fpr, tpr, thr = roc_curve(y_true[indices], y_pred[indices])
                auc += roc_auc_score(y_true, y_pred)
                
                # Find the optimal threshold: where the sum of FPR and TPR is closest to 1
                optimal_idx = np.argmin(np.abs(fpr + tpr - 1))
                optimal_threshold = thr[optimal_idx]
                # Binarize predictions based on the optimal threshold
                y_pred_binarized = (y_pred >= optimal_threshold).astype(int)

                # Calculate Balanced Accuracy
                balanced_accuracy += balanced_accuracy_score(y_true, y_pred_binarized)
            
            auc = auc / n_classes
            balanced_accuracy = balanced_accuracy / n_classes

        else:
            fpr, tpr, thr = roc_curve(y_true, y_pred, pos_label=1)
            auc = roc_auc_score(y_true, y_pred)

            # Find the optimal threshold: where the sum of FPR and TPR is closest to 1
            # "Balanced accuracy is symmetric. That is, the metric
            # assigns equal cost to false-positives and to false-negatives."
            # - LiRA https://arxiv.org/pdf/2112.03570.pdf
            optimal_idx = np.argmin(np.abs(fpr + tpr - 1))
            optimal_threshold = thr[optimal_idx]

            # Binarize predictions based on the optimal threshold
            y_pred_binarized = (y_pred >= optimal_threshold).astype(int)

            # Calculate Balanced Accuracy
            balanced_accuracy = balanced_accuracy_score(y_true, y_pred_binarized)
        
        print(f"AUC {method_name}: {auc*100:.2f}, Acc {balanced_accuracy * 100:.2f}")
        if method_name in auroc_all:
            auroc_all[method_name] += [auc]
            bal_acc_all[method_name] += [balanced_accuracy]
            tpr_m1[method_name] += [find_tpr_at_fpr(tpr, fpr, target_fpr=1e-2)]
            tpr_m2[method_name] += [find_tpr_at_fpr(tpr, fpr, target_fpr=1e-3)]
            tpr_m3[method_name] += [find_tpr_at_fpr(tpr, fpr, target_fpr=1e-4)]
        else:
            auroc_all[method_name] = [auc]
            bal_acc_all[method_name] = [balanced_accuracy]
            tpr_m1[method_name] = [find_tpr_at_fpr(tpr, fpr, target_fpr=1e-2)]
            tpr_m2[method_name] = [find_tpr_at_fpr(tpr, fpr, target_fpr=1e-3)]
            tpr_m3[method_name] = [find_tpr_at_fpr(tpr, fpr, target_fpr=1e-4)]


=================== Seed 64 ======================
AUC Curv ZO NLL: 93.55, Acc 84.62
AUC Curv ZO LR: 90.19, Acc 80.62
AUC Carlini et al.: 89.08, Acc 81.63
AUC Yeom et al.: 82.14, Acc 76.37
AUC Sablayrolles et al.: 81.32, Acc 70.41
AUC Watson et al.: 72.22, Acc 63.04
AUC Ye et al.: 91.03, Acc 80.82
AUC Song et al.: 82.24, Acc 75.53
=================== Seed 65 ======================
AUC Curv ZO NLL: 93.66, Acc 84.62
AUC Curv ZO LR: 90.13, Acc 80.48
AUC Carlini et al.: 88.88, Acc 81.66
AUC Yeom et al.: 82.48, Acc 76.73
AUC Sablayrolles et al.: 81.44, Acc 70.59
AUC Watson et al.: 71.74, Acc 62.79
AUC Ye et al.: 90.99, Acc 80.97
AUC Song et al.: 82.63, Acc 75.95
=================== Seed 66 ======================
AUC Curv ZO NLL: 93.23, Acc 84.17
AUC Curv ZO LR: 90.10, Acc 80.36
AUC Carlini et al.: 88.70, Acc 81.37
AUC Yeom et al.: 81.72, Acc 75.79
AUC Sablayrolles et al.: 80.55, Acc 69.65
AUC Watson et al.: 71.01, Acc 62.30
AUC Ye et al.: 90.62, Acc 80.40
AUC Song et al.: 81.97, Acc 75.25


In [9]:
for k, v in auroc_all.items():
    auc_for_k = np.array(v)
    bal_acc_for_k = np.array(bal_acc_all[k])

    print(f"{k}, auroc {auc_for_k.mean() * 100:.2f} $\pm$ {auc_for_k.std()* 100:.2f}, bal {bal_acc_for_k.mean()* 100:.2f} $\pm$ {bal_acc_for_k.std()* 100:.2f}")    

Curv ZO NLL, auroc 93.48 $\pm$ 0.18, bal 84.47 $\pm$ 0.21
Curv ZO LR, auroc 90.14 $\pm$ 0.04, bal 80.48 $\pm$ 0.11
Carlini et al., auroc 88.89 $\pm$ 0.16, bal 81.55 $\pm$ 0.13
Yeom et al., auroc 82.11 $\pm$ 0.31, bal 76.29 $\pm$ 0.39
Sablayrolles et al., auroc 81.11 $\pm$ 0.39, bal 70.22 $\pm$ 0.41
Watson et al., auroc 71.66 $\pm$ 0.50, bal 62.71 $\pm$ 0.31
Ye et al., auroc 90.88 $\pm$ 0.19, bal 80.73 $\pm$ 0.24
Song et al., auroc 82.28 $\pm$ 0.27, bal 75.58 $\pm$ 0.29


In [10]:
for k, v in tpr_m1.items():
    tpr_1 = np.array(v)
    tpr_2 = np.array(tpr_m2[k])
    tpr_3 = np.array(tpr_m3[k])

    print(f"{k}, top 1% {tpr_1.mean() * 100:.2f} $\pm$ {tpr_1.std()* 100:.2f}, tpr 0.1%{tpr_2.mean()* 100:.2f} $\pm$ {tpr_2.std()* 100:.2f}") 
    print(f"{k}, top 0.01% {tpr_3.mean() * 100:.2f} $\pm$ {tpr_3.std()* 100:.2f}") 

Curv ZO NLL, top 1% 41.54 $\pm$ 0.88, tpr 0.1%7.75 $\pm$ 1.45
Curv ZO NLL, top 0.01% 0.00 $\pm$ 0.00
Curv ZO LR, top 1% 35.54 $\pm$ 0.49, tpr 0.1%23.44 $\pm$ 1.19
Curv ZO LR, top 0.01% 5.23 $\pm$ 7.40
Carlini et al., top 1% 21.78 $\pm$ 0.64, tpr 0.1%15.80 $\pm$ 0.49
Carlini et al., top 0.01% 6.56 $\pm$ 1.13
Yeom et al., top 1% 0.94 $\pm$ 0.06, tpr 0.1%0.07 $\pm$ 0.01
Yeom et al., top 0.01% 0.01 $\pm$ 0.00
Sablayrolles et al., top 1% 21.05 $\pm$ 0.38, tpr 0.1%10.50 $\pm$ 0.59
Sablayrolles et al., top 0.01% 4.30 $\pm$ 0.53
Watson et al., top 1% 14.86 $\pm$ 0.14, tpr 0.1%6.62 $\pm$ 0.23
Watson et al., top 0.01% 2.95 $\pm$ 0.18
Ye et al., top 1% 34.72 $\pm$ 1.86, tpr 0.1%6.63 $\pm$ 0.52
Ye et al., top 0.01% 2.86 $\pm$ 0.42
Song et al., top 1% 2.15 $\pm$ 0.74, tpr 0.1%1.21 $\pm$ 0.37
Song et al., top 0.01% 1.21 $\pm$ 0.37
